Highest Cost Orders

Find the customer with the highest daily total order cost between 2019-02-01 to 2019-05-01. If customer had more than one order on a certain day, sum the order costs on daily basis. Output customer's first name, total cost of their items, and the date.


For simplicity, you can assume that every first name in the dataset is unique.

In [1]:
import pandas as pd
import numpy as np

In [10]:
customers = pd.read_excel("../CSV/customers.xlsx")
customers.head(3)

,Tаблица 1,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,id,first_name,last_name,city,address,phone_number
1,8,John,Joseph,San Francisco,NaN,928-386-8164
2,7,Jill,Michael,Austin,NaN,813-297-0692


In [11]:
orders = pd.read_excel("../CSV/orders.xlsx")
orders.head(3)

,Tаблица 1,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,id,cust_id,order_date,order_details,total_order_cost
1,1,3,2019-03-04,Coat,100
2,2,3,2019-03-01,Shoes,80


In [12]:
row_1_customers = customers.iloc[:1]
row_1_orders = orders.iloc[:1]
print(row_1_customers)
print(row_1_orders)

  Tаблица 1  Unnamed: 1 Unnamed: 2 Unnamed: 3 Unnamed: 4    Unnamed: 5
0        id  first_name  last_name       city    address  phone_number
  Tаблица 1 Unnamed: 1  Unnamed: 2     Unnamed: 3        Unnamed: 4
0        id    cust_id  order_date  order_details  total_order_cost


In [13]:
dic_customers = {}
for i, j in row_1_customers.items():
    dic_customers[i] = j.values[0]
dic_customers

{'Tаблица 1': 'id',
 'Unnamed: 1': 'first_name',
 'Unnamed: 2': 'last_name',
 'Unnamed: 3': 'city',
 'Unnamed: 4': 'address',
 'Unnamed: 5': 'phone_number'}

In [14]:
dic_orders = {}
for i, j in row_1_orders.items():
    dic_orders[i] = j.values[0]
dic_orders

{'Tаблица 1': 'id',
 'Unnamed: 1': 'cust_id',
 'Unnamed: 2': 'order_date',
 'Unnamed: 3': 'order_details',
 'Unnamed: 4': 'total_order_cost'}

In [15]:
customers.rename(columns=dic_customers, inplace=True)
orders.rename(columns=dic_orders, inplace=True)
customers.drop(0, inplace=True)
orders.drop(0, inplace=True)
print(customers.head(3))
print(orders.head(3))

  id first_name last_name           city address  phone_number
1  8       John    Joseph  San Francisco     NaN  928-386-8164
2  7       Jill   Michael         Austin     NaN  813-297-0692
3  4    William    Daniel         Denver     NaN  813-368-1200
  id cust_id  order_date order_details total_order_cost
1  1       3  2019-03-04          Coat              100
2  2       3  2019-03-01         Shoes               80
3  3       3  2019-03-07         Skirt               30


In [16]:
merge = pd.merge(customers, orders, left_on="id", right_on="cust_id")
merge.head(3)

,id_x,first_name,last_name,city,address,phone_number,id_y,cust_id,order_date,order_details,total_order_cost
0,7,Jill,Michael,Austin,NaN,813-297-0692,4,7,2019-02-01,Coat,25
1,7,Jill,Michael,Austin,NaN,813-297-0692,5,7,2019-03-10,Shoes,80
2,7,Jill,Michael,Austin,NaN,813-297-0692,17,7,2019-04-19,Suit,150


In [17]:
merge["order_date"] = merge["order_date"].apply(pd.to_datetime)
merge.head(3)

,id_x,first_name,last_name,city,address,phone_number,id_y,cust_id,order_date,order_details,total_order_cost
0,7,Jill,Michael,Austin,NaN,813-297-0692,4,7,2019-02-01,Coat,25
1,7,Jill,Michael,Austin,NaN,813-297-0692,5,7,2019-03-10,Shoes,80
2,7,Jill,Michael,Austin,NaN,813-297-0692,17,7,2019-04-19,Suit,150


In [19]:
order_date = merge[merge["order_date"].between("2019-02-1", "2019-05-1")]
order_date.head(3)

,id_x,first_name,last_name,city,address,phone_number,id_y,cust_id,order_date,order_details,total_order_cost
0,7,Jill,Michael,Austin,NaN,813-297-0692,4,7,2019-02-01,Coat,25
1,7,Jill,Michael,Austin,NaN,813-297-0692,5,7,2019-03-10,Shoes,80
2,7,Jill,Michael,Austin,NaN,813-297-0692,17,7,2019-04-19,Suit,150


In [20]:
result = (
    order_date.groupby(["first_name", "order_date"])["total_order_cost"]
    .sum()
    .to_frame("max_cost")
    .reset_index()
)
result

,first_name,order_date,max_cost
0,Eva,2019-03-11,20
1,Farida,2019-03-01,80
2,Farida,2019-03-04,100
3,Farida,2019-03-07,30
4,Farida,2019-04-19,50
5,Henry,2019-02-01,80
6,Jill,2019-02-01,25
7,Jill,2019-03-10,80
8,Jill,2019-04-01,50
9,Jill,2019-04-02,30


In [21]:
result = result[result.max_cost == result.max_cost.max()]
result

,first_name,order_date,max_cost
12,Jill,2019-04-19,275


In [22]:
result["order_date"] = pd.to_datetime(
    result["order_date"], format="%Y-%m-%d"
).dt.strftime("%Y-%m-%d")
result

/var/folders/8j/skd2y7rd5k752cwx0v97h73c0000gn/T/ipykernel_93488/2279680633.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result["order_date"] = pd.to_datetime(


,first_name,order_date,max_cost
12,Jill,2019-04-19,275


Solution Walkthrough
This question is asking us to find the customer with the highest daily total order cost between a specific time period. We are given two tables: customers and orders.

To solve this problem, we will first merge the two tables on the id column from customers and the cust_id column from orders. Then, we will filter the merged table to only include rows with order_date between "2019-02-01" and "2019-05-01". Next, we group the filtered table by first_name and order_date and sum up the total_order_cost for each group. After that, we reset the index of the grouped table and rename the sum column to "max_cost". Finally, we filter the table to only include rows where the max_cost is equal to the maximum value of max_cost and format the order_date column to look like "YYYY-MM-DD".

Let's break down the steps in detail.

Understanding The Data
We have two tables: customers and orders.

The customers table has columns id, first_name, and last_name. The id column is the unique identifier for each customer.

The orders table has columns cust_id, order_date, and total_order_cost. The cust_id column is the foreign key that references the id column in the customers table.

The Problem Statement
We need to find the customer with the highest daily total order cost between 2019-02-01 to 2019-05-01. If a customer has more than one order on a certain day, we need to sum the order costs on a daily basis. We need to output the customer's first name, total cost of their items, and the date.

Breaking Down The Code
Let's break down the code step by step:

import pandas as pd
import numpy as np

# Merge the customers and orders tables on the id and cust_id columns
merge = pd.merge(customers, orders, left_on="id", right_on="cust_id")
We import the necessary libraries and then merge the customers and orders tables using the merge function from the pandas library. We specify the left_on parameter as "id" and the right_on parameter as "cust_id" to indicate the columns to merge on.

# Convert the order_date column to datetime format
merge["order_date"] = merge["order_date"].apply(pd.to_datetime)
We convert the "order_date" column in the merged table to datetime format using the apply method and the pd.to_datetime function.

# Filter the merged table to only include rows with order_date between "2019-02-01" and "2019-05-01"
order_date = merge[
    merge["order_date"].between("2019-02-01", "2019-05-01")
]
We filter the merged table to only include rows where the "order_date" is between "2019-02-01" and "2019-05-01" using the between method.

# Group the filtered table by first_name and order_date, sum the total_order_cost for each group,
# rename the sum column to "max_cost", and reset the index
result = (
    order_date.groupby(["first_name", "order_date"])[
        "total_order_cost"
    ]
    .sum()
    .to_frame("max_cost")
    .reset_index()
)
We group the filtered table by "first_name" and "order_date", sum up the "total_order_cost" for each group using the sum method, and convert the resulting series to a dataframe using the to_frame method. Then, we rename the sum column to "max_cost" using the rename method and reset the index using the reset_index method.

# Filter the result table to only include rows where max_cost is equal to the maximum value of max_cost
result = result[result.max_cost == result.max_cost.max()]
We filter the resulting table to only include rows where the "max_cost" is equal to the maximum value of "max_cost" using boolean indexing.

# Format the order_date column to look like "YYYY-MM-DD"
result["order_date"] = pd.to_datetime(
    result["order_date"], format="%Y-%m-%d"
).dt.strftime("%Y-%m-%d")
We convert the "order_date" column in the resulting table to datetime format using the pd.to_datetime function with the specified format "%Y-%m-%d". Then, we format the datetime objects to look like "YYYY-MM-DD" using the dt.strftime method.

Bringing It All Together
import pandas as pd
import numpy as np

# Merge the customers and orders tables on the id and cust_id columns
merge = pd.merge(customers, orders, left_on="id", right_on="cust_id")

# Convert the order_date column to datetime format
merge["order_date"] = merge["order_date"].apply(pd.to_datetime)

# Filter the merged table to only include rows with order_date between "2019-02-01" and "2019-05-01"
order_date = merge[
    merge["order_date"].between("2019-02-01", "2019-05-01")
]

# Group the filtered table by first_name and order_date, sum the total_order_cost for each group,
# rename the sum column to "max_cost", and reset the index
result = (
    order_date.groupby(["first_name", "order_date"])[
        "total_order_cost"
    ]
    .sum()
    .to_frame("max_cost")
    .reset_index()
)

# Filter the result table to only include rows where max_cost is equal to the maximum value of max_cost
result = result[result.max_cost == result.max_cost.max()]

# Format the order_date column to look like "YYYY-MM-DD"
result["order_date"] = pd.to_datetime(
    result["order_date"], format="%Y-%m-%d"
).dt.strftime("%Y-%m-%d")
Conclusion
In this walkthrough, we learned how to find the customer with the highest daily total order cost between a specific time period. We merged two tables, filtered the data based on order dates, grouped the data by customer and order date, and finally filtered the highest daily total order cost.